# Percy Mukwenya's Capstone Notebook

#### This is my jupyter notebook for the data science capstone

## 1. Obtaining data from Wikepidea

In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [4]:
postal_codes = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

soup = BeautifulSoup(postal_codes,'lxml')

### Extracting the table from entire html page


In [9]:
table_columns = ['PostalCode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = table_columns)
table = soup.find('table', {'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=table_columns)
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

df1 = df.reset_index()

#print(df.head(5))
#print('***')
#print(df.tail(5))

In [6]:
df1.info()
df1.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 4 columns):
index           210 non-null int64
PostalCode      210 non-null object
Borough         210 non-null object
Neighborhood    210 non-null object
dtypes: int64(1), object(3)
memory usage: 6.6+ KB


(210, 4)

In [7]:
df2= df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))

df2.info()
df2.shape
df2.loc[df2['Neighborhood']=="Not assigned",'Neighborhood']=df2.loc[df2['Neighborhood']=="Not assigned",'Borough']

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, M1B to M9W
Data columns (total 2 columns):
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(2)
memory usage: 2.4+ KB


In [8]:
df3 = df2.reset_index()

### Cleaning and grouping neighborhoods in same postal code

In [55]:
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [57]:
df3.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [58]:
df3.shape

(103, 3)